In [ ]:
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Run
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.environment import Environment
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.compute import AksCompute
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.webservice import AksEndpoint
from azureml.core.model import Model
from azureml.core.compute import AksCompute
from sklearn.linear_model   import LogisticRegression
from sklearn.preprocessing  import StandardScaler
from sklearn.ensemble       import RandomForestClassifier
from sklearn.metrics        import accuracy_score
from sklearn.model_selection import train_test_split
import azureml.core
from azureml.core import Run
from azureml.core import Workspace
from azureml.core import Experiment
import pandas as pd
import numpy as np
import pickle
import joblib
import os

In [ ]:
ws = Workspace.from_config()
experiment = Experiment(workspace=ws, name="azureml-iris-experiment")

In [ ]:
# Load Data for training
iris_df = pd.read_csv('./data/iris.csv')
iris_df.head()

In [ ]:
#Label Encoding (Gender)
iris_df['species'] = iris_df.species.map( {'setosa': 0, 'virginica': 1, 'versicolor': 2} ).astype(int)

In [ ]:
# Select Feature and Target
Feature = iris_df.drop(columns=['species'], axis=1)
Target  = iris_df['species']

In [ ]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(Feature, Target, test_size=0.2, random_state=42)

## Train a model

In [ ]:
estimators = [5, 10, 20, 50, 100]

for estimator in estimators:
    run = experiment.start_logging()
    run.log("estimator_value", estimator)
    
    # Random Forest classifier
    classifier = RandomForestClassifier(n_estimators=estimator, random_state=42)
    classifier.fit(X_train, y_train)
    
    y_pred = classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    run.log('Accuracy', accuracy)
    
    model_name = "rf_model_est_" + str(estimator) + ".pkl"
    filename = "outputs/" + model_name
    
    joblib.dump(value=classifier, filename=f'outputs/rf_model_est_{str(estimator)}.pkl')
    
    run.upload_file(name=model_name, path_or_stream=filename)
    run.complete()

## Get the best model

In [ ]:
best_acc_runid = None
best_acc = None

best_modeldf = pd.DataFrame()

for run in experiment.get_runs():
    run_metrics = run.get_metrics()
    run_details = run.get_details()
#     best_modeldf = best_modeldf.append([run_metrics])
#     print(run_details)
# Choose best estomator
    
    # each logged metric becomes a key in this returned dict
    try:
        run_acc = run_metrics["Accuracy"]
        run_id = run_details["runId"]
    except:
        run_acc = 0
        run_id  = 999
    
    if best_acc is None:
        best_acc = run_acc
        best_acc_runid = run_id
    else:
        if run_acc > best_acc:
            best_acc = run_acc
            best_acc_runid = run_id

print("Best run_id: " + best_acc_runid)
print("Best run_id accuracy: " + str(best_acc))

In [ ]:
# Choose Best Estimator
best_run = Run(experiment=experiment, run_id=best_acc_runid)
best_run
print(best_run.get_file_names())

In [ ]:
# best_run.download_file(name="rf_model_est_100.pkl")

In [ ]:
# run.complete()

## Register the model

In [ ]:
# register model
model = run.register_model(model_name='sklearn_iris_model',
                           model_path='outputs/rf_model_est_5.pkl')
print(model.name, model.id, model.version, sep='\t')

## Scoring

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'rf_model_est_5.pkl')
    model = joblib.load(model_path)

def run(raw_data):
    predict_map = {0:'setosa', 1:'virginica', 2:'versicolor'}
    data = np.array(json.loads(raw_data)['data']).reshape(-1, 1)
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return predict_map[y_hat.tolist()[0]]

## Create ACI Instance & Deploy

In [ ]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "IRIS",  "method" : "sklearn"}, 
                                               description='Predict IRIS with sklearn')

In [ ]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, id='sklearn_iris_model:1') # We can get this from model section


myenv = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version="10")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

service_name = 'test-sklearn-iris-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

In [ ]:
print(service.scoring_uri)

In [ ]:
## Sending Raw HTTPS request
import requests

# send a random row from the test set to score
input_data = "{\"data\": [" + str(list(X_test.iloc[10])) + "]}"

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
#print("input data:", input_data)
print("label:", y_test.iloc[10])
print("prediction:", resp.text)

In [ ]:
X_test.iloc[10]

In [ ]:
"{\"data\": [" + str(list(X_test.iloc[10])) + "]}"